In [1]:
# set this path to learnMSA 
learn_msa_path = "../../learnMSA"

import sys, os
sys.path.append(learn_msa_path) 
sys.path.append("../../bin")
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
from BCBio import GFF
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import tensorflow.keras as keras
from eval_model_class import PredictionGTF
from models import lstm_model, custom_cce_f1_loss

2024-07-20 08:29:05.755801: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-20 08:29:05.802750: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-20 08:29:06.776396: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-20 08:29:06.777858: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [2]:
batch_size = 20
seq_len = 9999
strand = '+'

emb=False
hmm_factor=1
trans_lstm=False

inp_data_dir = 'inp/'
out_dir = 'test_train/'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

genome_path = f'{inp_data_dir}/genome.fa'
annot_path= f'{inp_data_dir}/annot.gtf'


pred_gtf = PredictionGTF( 
     seq_len=seq_len, 
    batch_size=batch_size,
    hmm=True, 
    emb=False, 
    num_hmm=1,
    hmm_factor=1,
    genome_path=genome_path,
    annot_path=annot_path, 
    softmask=True, strand=strand,
)

# load input data x_seq 
x_seq, y_seq, _ = pred_gtf.load_inp_data(    
    strand=strand, 
    chunk_coords=True, softmask=True, pad=False
)


In [3]:
# see lstm_model documentation for more arguments 
config = {
    "num_epochs": 10,
    "stride": 0,
    "units": 100,
    "filter_size": 32,
    "numb_lstm": 2,
    "numb_conv": 3,
    "dropout_rate": 0.0,
    "hmm_dense": 32,
    "pool_size": 9,
    "lr": 1e-4,
    "batch_size": batch_size,
    "w_size": seq_len,
    'output_size': 15,
}

relevant_keys = ['units', 'filter_size', 'kernel_size', 
                     'numb_conv', 'numb_lstm', 'dropout_rate', 
                     'pool_size', 'stride',  
                     'output_size', 'multi_loss']

relevant_args = {key: config[key] for key in relevant_keys if key in config}
model = lstm_model(**relevant_args)
adam = Adam(learning_rate=config["lr"])
f1loss = custom_cce_f1_loss(2, batch_size=config["batch_size"])


model.compile(loss=f1loss, optimizer=adam, metrics=['accuracy'])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, None, 6)]    0           []                               
                                                                                                  
 initial_conv (Conv1D)          (None, None, 32)     608         ['main_input[0][0]']             
                                                                                                  
 layer_normalization1 (LayerNor  (None, None, 32)    64          ['initial_conv[0][0]']           
 malization)                                                                                      
                                                                                                  
 conv_1 (Conv1D)                (None, None, 32)     9248        ['layer_normalization1[0][0

In [ ]:
model.fit(x=x_seq, y=y_seq, 
          epochs=config["num_epochs"], 
          batch_size=config["batch_size"])

Epoch 1/10
 68/548 [==>...........................] - ETA: 6:29 - loss: 3.9613 - accuracy: 0.5674